# Creating a Sentiment Analysis Web App
## Using PyTorch and SageMaker

_Deep Learning Nanodegree Program | Deployment_

---

Now that we have a basic understanding of how SageMaker works we will try to use it to construct a complete project from end to end. Our goal will be to have a simple web page which a user can use to enter a movie review. The web page will then send the review off to our deployed model which will predict the sentiment of the entered review.

## Instructions

Some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this notebook. You will not need to modify the included code beyond what is requested. Sections that begin with '**TODO**' in the header indicate that you need to complete or implement some portion within them. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `# TODO: ...` comment. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions for you to answer which relate to the task and your implementation. Each section where you will answer a question is preceded by a '**Question:**' header. Carefully read each question and provide your answer below the '**Answer:**' header by editing the Markdown cell.

> **Note**: Code and Markdown cells can be executed using the **Shift+Enter** keyboard shortcut. In addition, a cell can be edited by typically clicking it (double-click for Markdown cells) or by pressing **Enter** while it is highlighted.

## General Outline

Recall the general outline for SageMaker projects using a notebook instance.

1. Download or otherwise retrieve the data.
2. Process / Prepare the data.
3. Upload the processed data to S3.
4. Train a chosen model.
5. Test the trained model (typically using a batch transform job).
6. Deploy the trained model.
7. Use the deployed model.

For this project, you will be following the steps in the general outline with some modifications. 

First, you will not be testing the model in its own step. You will still be testing the model, however, you will do it by deploying your model and then using the deployed model by sending the test data to it. One of the reasons for doing this is so that you can make sure that your deployed model is working correctly before moving forward.

In addition, you will deploy and use your trained model a second time. In the second iteration you will customize the way that your trained model is deployed by including some of your own code. In addition, your newly deployed model will be used in the sentiment analysis web app.

## Step 1: Downloading the data

As in the XGBoost in SageMaker notebook, we will be using the [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/)

> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.

In [1]:
#%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

--2019-04-29 12:10:39--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  24.6MB/s    in 3.9s    

2019-04-29 12:10:43 (20.7 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



## Step 2: Preparing and Processing the data

Also, as in the XGBoost notebook, we will be doing some initial data processing. The first few steps are the same as in the XGBoost example. To begin with, we will read in each of the reviews and combine them into a single input structure. Then, we will split the dataset into a training set and a testing set.

In [1]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [2]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


Now that we've read the raw training and testing data from the downloaded dataset, we will combine the positive and negative reviews and shuffle the resulting records.

In [3]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [4]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


Now that we have our training and testing sets unified and prepared, we should do a quick check and see an example of the data our model will be trained on. This is generally a good idea as it allows you to see how each of the further processing steps affects the reviews and it also ensures that the data has been loaded correctly.

In [5]:
print(train_X[100])
print(train_y[100])

I was aware of Rohmer's admiration for the late works of the ones he considered like great cineasts, and that normal spectators generally considered as artistic failures (as Renoir's or Chaplin's very last movies ; yes, the "politique des auteurs" also has its dark side). But with "Les amours d'Astrée et de Céladon", it's as if Rohmer himself wanted, for what may be his last movie, to perpetuate this tradition of great directors, who made a last senile movie, by adapting Urfé's "L'astrée", with ridiculous aesthetic codes, witch just look like a parody of Rosselini's last movies (the ones he made for TV from Descartes or Marx's lives).<br /><br />In his version of "Perceval", Rohmer refused to film real landscapes in order to give a re-transcription of what may have been a middle age classical representation of things. The director apparently changed his mind when the XVII century is involved, and films actors, dressed like 1600's peasants reciting their antic text surrounded by contemp

The first step in processing the reviews is to make sure that any html tags that appear should be removed. In addition we wish to tokenize our input, that way words such as *entertained* and *entertaining* are considered the same with regard to sentiment analysis.

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

nltk.download("stopwords", quiet=True)
stemmer = PorterStemmer()

In [7]:
import re
from bs4 import BeautifulSoup

def review_to_words(review):
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

The `review_to_words` method defined above uses `BeautifulSoup` to remove any html tags that appear and uses the `nltk` package to tokenize the reviews. As a check to ensure we know how everything is working, try applying `review_to_words` to one of the reviews in the training set.

In [8]:
# TODO: Apply review_to_words to a review (train_X[100] or any other review)
review_to_words(train_X[100])

['awar',
 'rohmer',
 'admir',
 'late',
 'work',
 'one',
 'consid',
 'like',
 'great',
 'cineast',
 'normal',
 'spectat',
 'gener',
 'consid',
 'artist',
 'failur',
 'renoir',
 'chaplin',
 'last',
 'movi',
 'ye',
 'politiqu',
 'de',
 'auteur',
 'also',
 'dark',
 'side',
 'le',
 'amour',
 'astr',
 'e',
 'et',
 'de',
 'c',
 'ladon',
 'rohmer',
 'want',
 'may',
 'last',
 'movi',
 'perpetu',
 'tradit',
 'great',
 'director',
 'made',
 'last',
 'senil',
 'movi',
 'adapt',
 'urf',
 'l',
 'astr',
 'e',
 'ridicul',
 'aesthet',
 'code',
 'witch',
 'look',
 'like',
 'parodi',
 'rosselini',
 'last',
 'movi',
 'one',
 'made',
 'tv',
 'descart',
 'marx',
 'live',
 'version',
 'percev',
 'rohmer',
 'refus',
 'film',
 'real',
 'landscap',
 'order',
 'give',
 'transcript',
 'may',
 'middl',
 'age',
 'classic',
 'represent',
 'thing',
 'director',
 'appar',
 'chang',
 'mind',
 'xvii',
 'centuri',
 'involv',
 'film',
 'actor',
 'dress',
 'like',
 '1600',
 'peasant',
 'recit',
 'antic',
 'text',
 'surroun

**Question:** Above we mentioned that `review_to_words` method removes html formatting and allows us to tokenize the words found in a review, for example, converting *entertained* and *entertaining* into *entertain* so that they are treated as though they are the same word. What else, if anything, does this method do to the input?

**Answer:** It also converts text to lower case, and removes stop words that do not have any decisive power to determine sentence sentiment.

The method below applies the `review_to_words` method to each of the reviews in the training and testing datasets. In addition it caches the results. This is because performing this processing step can take a long time. This way if you are unable to complete the notebook in the current session, you can come back without needing to process the data a second time.

In [9]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [10]:
%%time
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Read preprocessed data from cache file: preprocessed_data.pkl
CPU times: user 758 ms, sys: 201 ms, total: 959 ms
Wall time: 960 ms


## Transform the data

In the XGBoost notebook we transformed the data from its word representation to a bag-of-words feature representation. For the model we are going to construct in this notebook we will construct a feature representation which is very similar. To start, we will represent each word as an integer. Of course, some of the words that appear in the reviews occur very infrequently and so likely don't contain much information for the purposes of sentiment analysis. The way we will deal with this problem is that we will fix the size of our working vocabulary and we will only include the words that appear most frequently. We will then combine all of the infrequent words into a single category and, in our case, we will label it as `1`.

Since we will be using a recurrent neural network, it will be convenient if the length of each review is the same. To do this, we will fix a size for our reviews and then pad short reviews with the category 'no word' (which we will label `0`) and truncate long reviews.

### (TODO) Create a word dictionary

To begin with, we need to construct a way to map words that appear in the reviews to integers. Here we fix the size of our vocabulary (including the 'no word' and 'infrequent' categories) to be `5000` but you may wish to change this to see how it affects the model.

> **TODO:** Complete the implementation for the `build_dict()` method below. Note that even though the vocab_size is set to `5000`, we only want to construct a mapping for the most frequently appearing `4998` words. This is because we want to reserve the special labels `0` for 'no word' and `1` for 'infrequent word'.

In [11]:
import numpy as np

def build_dict(data, vocab_size = 5000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    
    # TODO: Determine how often each word appears in `data`. Note that `data` is a list of sentences and that a
    #       sentence is a list of words.
    
    word_count = {} # A dict storing the words that appear in the reviews along with how often they occur
    for sentence in data:
        for word in sentence:
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1
    
    # TODO: Sort the words found in `data` so that sorted_words[0] is the most frequently appearing word and
    #       sorted_words[-1] is the least frequently appearing word.
    
    sorted_words = sorted(word_count, key = word_count.get, reverse=True)
    
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_count, sorted_words, word_dict

In [12]:
word_count, sorted_words, word_dict = build_dict(train_X)

**Question:** What are the five most frequently appearing (tokenized) words in the training set? Does it makes sense that these words appear frequently in the training set?

**Answer:** ['time', 'like', 'one', 'film', 'movi'] totally makes sence, since film and movi are realated to the subject of the dataset and are likely to be present in every review. Other words from the top-5 are pretty generic words too.

In [13]:
# TODO: Use this space to determine the five most frequently appearing words in the training set.
sorted_words[:5]

['movi', 'film', 'one', 'like', 'time']

In [14]:
word_count

{'worst': 2731,
 'movi': 51695,
 'ever': 5999,
 'seen': 6681,
 'go': 9305,
 'get': 14141,
 'leav': 2272,
 'tape': 369,
 '4': 1372,
 'stuck': 350,
 'consid': 1585,
 'masochist': 45,
 'afghanistan': 36,
 'come': 6747,
 'guy': 4339,
 'idiot': 464,
 'forgot': 178,
 'hide': 419,
 'sanskrit': 2,
 'billboard': 25,
 'thought': 3773,
 'lead': 2684,
 'actor': 6876,
 'georg': 933,
 'calil': 1,
 'particularli': 1079,
 'inept': 187,
 'apart': 995,
 'bad': 9344,
 'act': 8794,
 'zealou': 11,
 'camera': 1888,
 'shake': 194,
 'use': 5123,
 'event': 1293,
 '9': 787,
 '11': 369,
 'reason': 3206,
 'make': 15207,
 'larson': 9,
 'lunat': 58,
 'implod': 5,
 'screen': 2799,
 'near': 843,
 'disgrac': 102,
 'irrever': 37,
 'victim': 816,
 'phone': 401,
 'call': 2792,
 'wife': 2140,
 'sarah': 192,
 'supposedli': 365,
 'one': 27741,
 'terrorist': 192,
 'held': 391,
 'plane': 380,
 'day': 4013,
 'appal': 163,
 'feel': 5300,
 'sick': 531,
 'cold': 577,
 'heart': 1688,
 'stunt': 286,
 'artisticli': 1,
 'shot': 2998,

In [15]:
word_dict

{'movi': 2,
 'film': 3,
 'one': 4,
 'like': 5,
 'time': 6,
 'good': 7,
 'make': 8,
 'charact': 9,
 'get': 10,
 'see': 11,
 'watch': 12,
 'stori': 13,
 'even': 14,
 'would': 15,
 'realli': 16,
 'well': 17,
 'scene': 18,
 'look': 19,
 'show': 20,
 'much': 21,
 'end': 22,
 'peopl': 23,
 'bad': 24,
 'go': 25,
 'great': 26,
 'also': 27,
 'first': 28,
 'love': 29,
 'think': 30,
 'way': 31,
 'act': 32,
 'play': 33,
 'made': 34,
 'thing': 35,
 'could': 36,
 'know': 37,
 'say': 38,
 'seem': 39,
 'work': 40,
 'plot': 41,
 'two': 42,
 'actor': 43,
 'year': 44,
 'come': 45,
 'mani': 46,
 'seen': 47,
 'take': 48,
 'want': 49,
 'life': 50,
 'never': 51,
 'littl': 52,
 'best': 53,
 'tri': 54,
 'man': 55,
 'ever': 56,
 'give': 57,
 'better': 58,
 'still': 59,
 'perform': 60,
 'find': 61,
 'feel': 62,
 'part': 63,
 'back': 64,
 'use': 65,
 'someth': 66,
 'director': 67,
 'actual': 68,
 'interest': 69,
 'lot': 70,
 'real': 71,
 'old': 72,
 'cast': 73,
 'though': 74,
 'live': 75,
 'star': 76,
 'enjoy': 7

In [16]:
word_count['tarzan']

292

In [17]:
word_dict['tarzan']

1656

In [18]:
'tarzan' in sorted_words

True

### Save `word_dict`

Later on when we construct an endpoint which processes a submitted review we will need to make use of the `word_dict` which we have created. As such, we will save it to a file now for future use.

In [19]:
data_dir = '../data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [20]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

### Transform the reviews

Now that we have our word dictionary which allows us to transform the words appearing in the reviews into integers, it is time to make use of it and convert our reviews to their integer sequence representation, making sure to pad or truncate to a fixed length, which in our case is `500`.

In [21]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [22]:
%%time
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

CPU times: user 3.17 s, sys: 128 ms, total: 3.3 s
Wall time: 3.3 s


In [23]:
test_X.shape, test_X_len.shape

((25000, 500), (25000,))

### Split train into train/val

In [ ]:
#TODO

As a quick check to make sure that things are working as intended, check to see what one of the reviews in the training set looks like after having been processeed. Does this look reasonable? What is the length of a review in the training set?

In [24]:
# Use this cell to examine one of the processed reviews to make sure everything is working as intended.
len(train_X[101])

500

In [25]:
train_X[101]

array([  30,   22, 4369,  160,  213,   55, 3183,   38,  307, 3223, 1914,
         29,   13, 2274, 1949,    1,  374,  831,    1,    1, 1660,  192,
          1,    7,    2,  531,   77, 1541, 2191,    2,  385, 3743,   17,
        184,  748,   23,   12,   94,  389,  319,    6,  200,   25,   12,
        135, 2921,    3,  251,    8,   62,    5,  528,  140, 3819, 1679,
       3819,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

**Question:** In the cells above we use the `preprocess_data` and `convert_and_pad_data` methods to process both the training and testing set. Why or why not might this be a problem?

**Answer:** This is not a problem since we use the same dictionary. There might be a problem if test dataset contains words that are not prosent in the dict. But we have accounted for this using an INFREQ token.

## Step 3: Upload the data to S3

As in the XGBoost notebook, we will need to upload the training dataset to S3 in order for our training code to access it. For now we will save it locally and we will upload to S3 later on.

### Save the processed training dataset locally

It is important to note the format of the data that we are saving as we will need to know it when we write the training code. In our case, each row of the dataset has the form `label`, `length`, `review[500]` where `review[500]` is a sequence of `500` integers representing the words in the review.

In [26]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

### Uploading the training data


Next, we need to upload the training data to the SageMaker default S3 bucket so that we can provide access to it while training our model.

In [27]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.predictor import csv_serializer


In [28]:
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

role = sagemaker.get_execution_role()

In [29]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

In [30]:
input_data

's3://sagemaker-us-east-2-680176548196/sagemaker/sentiment_rnn'

**NOTE:** The cell above uploads the entire contents of our data directory. This includes the `word_dict.pkl` file. This is fortunate as we will need this later on when we create an endpoint that accepts an arbitrary review. For now, we will just take note of the fact that it resides in the data directory (and so also in the S3 training bucket) and that we will need to make sure it gets saved in the model directory.

## Step 4: Build and Train the PyTorch Model

In the XGBoost notebook we discussed what a model is in the SageMaker framework. In particular, a model comprises three objects

 - Model Artifacts,
 - Training Code, and
 - Inference Code,
 
each of which interact with one another. In the XGBoost example we used training and inference code that was provided by Amazon. Here we will still be using containers provided by Amazon with the added benefit of being able to include our own custom code.

We will start by implementing our own neural network in PyTorch along with a training script. For the purposes of this project we have provided the necessary model object in the `model.py` file, inside of the `train` folder. You can see the provided implementation by running the cell below.

In [31]:
!pygmentize train/model.py

import torch.nn as nn

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig

The important takeaway from the implementation provided is that there are three parameters that we may wish to tweak to improve the performance of our model. These are the embedding dimension, the hidden dimension and the size of the vocabulary. We will likely want to make these parameters configurable in the training script so that if we wish to modify them we do not need to modify the script itself. We will see how to do this later on. To start we will write some of the training code in the notebook so that we can more easily diagnose any issues that arise.

First we will load a small portion of the training data set to use as a sample. It would be very time consuming to try and train the model completely in the notebook as we do not have access to a gpu and the compute instance that we are using is not particularly powerful. However, we can work on a small bit of the data to get a feel for how our training script is behaving.

In [32]:
import torch
import torch.utils.data

# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

In [33]:
train_sample.head()

,0,1,2,3,4,5,6,7,8,9,...,492,493,494,495,496,497,498,499,500,501
0,0,71,174,2,56,47,25,10,219,1377,...,0,0,0,0,0,0,0,0,0,0
1,1,53,1,150,43,286,562,81,10,71,...,0,0,0,0,0,0,0,0,0,0
2,0,56,94,190,1805,1124,2,49,1,25,...,0,0,0,0,0,0,0,0,0,0
3,1,377,12,2276,1420,2276,1,197,11,516,...,0,0,0,0,0,0,0,0,0,0
4,0,80,2180,948,1,26,378,2,130,10,...,0,0,0,0,0,0,0,0,0,0


In [34]:
train_sample.shape

(250, 502)

### (TODO) Writing the training method

Next we need to write the training code itself. This should be very similar to training methods that you have written before to train PyTorch models. We will leave any difficult aspects such as model saving / loading and parameter loading until a little later.

In [35]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            # TODO: Complete this train method to train the model provided.
            output = model(batch_X)
            loss = loss_fn(output, batch_y)
            
            loss.backward()
            optimizer.step()
            #=================
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

Supposing we have the training method above, we will test that it is working by writing a bit of code in the notebook that executes our training method on the small sample training set that we loaded earlier. The reason for doing this in the notebook is so that we have an opportunity to fix any errors that arise early when they are easier to diagnose.

In [36]:
import torch.optim as optim
from train.model import LSTMClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

In [38]:
device

device(type='cpu')

In [39]:
epochs=10
train(model, train_sample_dl, epochs, optimizer, loss_fn, device)

Epoch: 1, BCELoss: 0.695549464225769
Epoch: 2, BCELoss: 0.6856090545654296
Epoch: 3, BCELoss: 0.6736214518547058
Epoch: 4, BCELoss: 0.6577884912490845
Epoch: 5, BCELoss: 0.6344348788261414
Epoch: 6, BCELoss: 0.5965411901473999
Epoch: 7, BCELoss: 0.5432931542396545
Epoch: 8, BCELoss: 0.5194726467132569
Epoch: 9, BCELoss: 0.4357926845550537
Epoch: 10, BCELoss: 0.43708168268203734


In order to construct a PyTorch model using SageMaker we must provide SageMaker with a training script. We may optionally include a directory which will be copied to the container and from which our training code will be run. When the training container is executed it will check the uploaded directory (if there is one) for a `requirements.txt` file and install any required Python libraries, after which the training script will be run.

### (TODO) Training the model

When a PyTorch model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained. Inside of the `train` directory is a file called `train.py` which has been provided and which contains most of the necessary code to train our model. The only thing that is missing is the implementation of the `train()` method which you wrote earlier in this notebook.

**TODO**: Copy the `train()` method written above and paste it into the `train/train.py` file where required.

The way that SageMaker passes hyperparameters to the training script is by way of arguments. These arguments can then be parsed and used in the training script. To see how this is done take a look at the provided `train/train.py` file.

In [31]:
use_instance = 'ml.p2.xlarge' # 'ml.p2.xlarge'  #'ml.m5.2xlarge' #  ml.p3.2xlarge',

In [40]:
static_hyperpameters=dict(
    lr=0.0001,
    epochs=15,
    batch_size=256,
    hidden_dim=100,
    embedding_dim=32,
    vocab_size=5000
)

In [41]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type=use_instance,
                    hyperparameters=static_hyperpameters)

In [42]:
estimator.fit({'training': input_data})

2019-05-28 10:42:24 Starting - Starting the training job...
2019-05-28 10:42:25 Starting - Launching requested ML instances......
2019-05-28 10:43:27 Starting - Preparing the instances for training......
2019-05-28 10:44:28 Downloading - Downloading input data...
2019-05-28 10:44:56 Training - Downloading the training image..
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-05-28 10:45:32,124 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-05-28 10:45:32,148 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-05-28 10:45:32,154 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-05-28 10:45:32,388 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2019-05-28 10:45:32,388 sagemaker-containers INFO     Generating setup.cfg
2019-05-28 10:45:32,388 sagemaker-containers 

Model loaded with embedding_dim 32, hidden_dim 100, vocab_size 5000.
Epoch: 1, BCELoss: 0.6891338113619356;
Epoch: 2, BCELoss: 0.6709218596925541;
Epoch: 3, BCELoss: 0.6591852635753398;
Epoch: 4, BCELoss: 0.6397979423707846;
Epoch: 5, BCELoss: 0.6478344864991247;
Epoch: 6, BCELoss: 0.6372441619026418;
Epoch: 7, BCELoss: 0.610834714709496;
Epoch: 8, BCELoss: 0.5689285388406442;
Epoch: 9, BCELoss: 0.5536262687979913;
Epoch: 10, BCELoss: 0.5461397146692082;
Epoch: 11, BCELoss: 0.5584469103083318;
Epoch: 12, BCELoss: 0.5364921667746135;
Epoch: 13, BCELoss: 0.4885812058132522;

2019-05-28 10:48:23 Uploading - Uploading generated training modelEpoch: 14, BCELoss: 0.45385624833252963;
Epoch: 15, BCELoss: 0.4545373712875405;
2019-05-28 10:48:21,825 sagemaker-containers INFO     Reporting training SUCCESS

2019-05-28 10:48:29 Completed - Training job completed
Billable seconds: 241


In [43]:
estimator

In [62]:
estimator.hyperparameters()

{'lr': '0.0001',
 'epochs': '15',
 'batch_size': '256',
 'hidden_dim': '100',
 'embedding_dim': '32',
 'vocab_size': '5000',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-2-680176548196/sagemaker-pytorch-2019-05-28-11-05-14-832/source/sourcedir.tar.gz"',
 'sagemaker_program': '"train.py"',
 'sagemaker_enable_cloudwatch_metrics': 'false',
 'sagemaker_container_log_level': '20',
 'sagemaker_job_name': '"sagemaker-pytorch-2019-05-28-11-05-14-832"',
 'sagemaker_region': '"us-east-2"'}

## Tuning

In [117]:
# As stated above, we use this utility method to construct the image name for the training container.
#container = get_image_uri(sagemaker_session.boto_region_name, 'sagemaker-pytorch-container')

# Now that we know which container to use, we can construct the estimator object.
#pt = sagemaker.estimator.Estimator(container, # The name of the training container
#                                    role,      # The IAM role to use (our current role in this case)
#                                    train_instance_count=1, # The number of instances to use for training
#                                    train_instance_type=use_instance, # The type of instance ot use for training
#                                    output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix),
#                                                                        # Where to save the output (the model artifacts)
#                                    sagemaker_session=session) # The current SageMaker session

In [118]:
#pt = estimator

In [116]:
#pt.set_hyperparameters(static_hyperpameters)

In [55]:
from sagemaker.tuner import HyperparameterTuner
from sagemaker.tuner import IntegerParameter, ContinuousParameter, CategoricalParameter

In [87]:
hyperparameter_ranges={
    'lr'      : ContinuousParameter(0.00001, 0.01, scaling_type='Logarithmic'),
    #'epochs': IntegerParameter([5,10,30]),
    'batch_size': CategoricalParameter([128, 256, 512]),
    'hidden_dim': CategoricalParameter([100, 200, 300]),
    'embedding_dim': CategoricalParameter([32, 64]),
    'vocab_size': IntegerParameter(2000, 20000),
}

In [88]:
pt_hyperparameter_tuner = HyperparameterTuner(
    estimator = pt, # The estimator object to use as the basis for the training jobs.
    objective_metric_name='bce_loss',
    metric_definitions = [
    {
        "Name": "bce_loss",
        "Regex": "BCELoss: (.*?);",
    }],
    objective_type = 'Minimize', # Whether we wish to minimize or maximize the metric.
    max_jobs = 40, # The total number of models to train
    max_parallel_jobs = 2, # The number of models to train in parallel
    hyperparameter_ranges = hyperparameter_ranges,
    early_stopping_type='Auto'
)

In [89]:
# This is a wrapper around the location of our train and validation data, to make sure that SageMaker
# knows our data is in csv format.
s3_input_train = sagemaker.s3_input(s3_data=input_data, content_type='csv')

#s3_input_validation = sagemaker.s3_input(s3_data=val_location, content_type='csv')

pt_hyperparameter_tuner.fit(
    {'training': s3_input_train, 
     #'validation': s3_input_validation
    } 
)

In [ ]:
#TODO split train/val

In [90]:
pt_hyperparameter_tuner.wait()

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

### Get the best job

In [94]:
pt_hyperparameter_tuner.best_training_job()

'sagemaker-pytorch-190528-1311-029-2cfdf7de'

In [97]:
pt_attached = sagemaker.estimator.Estimator.attach(pt_hyperparameter_tuner.best_training_job())

2019-05-28 15:00:13 Starting - Preparing the instances for training
2019-05-28 15:00:13 Downloading - Downloading input data
2019-05-28 15:00:13 Training - Training image download completed. Training in progress.
2019-05-28 15:00:13 Uploading - Uploading generated training model
2019-05-28 15:00:13 Completed - Training job completedbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-05-28 14:55:05,758 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-05-28 14:55:05,760 sagemaker-containers INFO     Failed to parse hyperparameter _tuning_objective_metric value bce_loss to Json.
Returning the value itself
2019-05-28 14:55:05,787 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-05-28 14:55:08,807 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-05-28 14:55:09,505 sagemaker-containers INFO     Module train doe

In [98]:
estimator = pt_attached

In [99]:
estimator.hyperparameters()

{'_tuning_objective_metric': 'bce_loss',
 'batch_size': '"128"',
 'embedding_dim': '"64"',
 'epochs': '15',
 'hidden_dim': '"100"',
 'lr': '0.00029474112786947386',
 'sagemaker_container_log_level': '20',
 'sagemaker_enable_cloudwatch_metrics': 'false',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"sagemaker-pytorch-2019-05-28-13-11-23-491"',
 'sagemaker_program': '"train.py"',
 'sagemaker_region': '"us-east-2"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-2-680176548196/sagemaker-pytorch-2019-05-28-13-11-23-491/source/sourcedir.tar.gz"',
 'vocab_size': '8074'}

In [1]:
# This does not work
#pt_hyperparameter_tuner.deploy(initial_instance_count=1, instance_type=use_instance)

## Step 5: Testing the model

As mentioned at the top of this notebook, we will be testing this model by first deploying it and then sending the testing data to the deployed endpoint. We will do this so that we can make sure that the deployed model is working correctly.

## Step 6: Deploy the model for testing

Now that we have trained our model, we would like to test it to see how it performs. Currently our model takes input of the form `review_length, review[500]` where `review[500]` is a sequence of `500` integers which describe the words present in the review, encoded using `word_dict`. Fortunately for us, SageMaker provides built-in inference code for models with simple inputs such as this.

There is one thing that we need to provide, however, and that is a function which loads the saved model. This function must be called `model_fn()` and takes as its only parameter a path to the directory where the model artifacts are stored. This function must also be present in the python file which we specified as the entry point. In our case the model loading function has been provided and so no changes need to be made.

**NOTE**: When the built-in inference code is run it must import the `model_fn()` method from the `train.py` file. This is why the training code is wrapped in a main guard ( ie, `if __name__ == '__main__':` )

Since we don't need to change anything in the code that was uploaded during training, we can simply deploy the current model as-is.

**NOTE:** When deploying a model you are asking SageMaker to launch an compute instance that will wait for data to be sent to it. As a result, this compute instance will continue to run until *you* shut it down. This is important to know since the cost of a deployed endpoint depends on how long it has been running for.

In other words **If you are no longer using a deployed endpoint, shut it down!**

**TODO:** Deploy the trained model.

In [115]:
# Deploy the trained model
# Attention - this does not work for hyperparameterTuner jobs!
#use_instance = 'ml.m4.xlarge'
#predictor = estimator.deploy(initial_instance_count=1, instance_type=use_instance)

In [101]:
model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='train.py',
                     source_dir='train',
#                     predictor_cls=StringPredictor
)

predictor = model.deploy(initial_instance_count=1, instance_type=use_instance)

---------------------------------------------------------------------------------------------------!

In [47]:
predictor

## Step 7 - Use the model for testing

Once deployed, we can read in the test data and send it off to our deployed model to get some results. Once we collect all of the results we can determine how accurate our model is.

In [102]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1)

In [103]:
# We split the data into chunks and send each chunk seperately, accumulating the results.

def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        predictions = np.append(predictions, predictor.predict(array))
    
    return predictions

In [104]:
%time
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.44 µs


In [105]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.7488

In [106]:
len(predictions)

25000

In [107]:
test_X.values.shape

(25000, 502)

**Question:** How does this model compare to the XGBoost model you created earlier? Why might these two models perform differently on this dataset? Which do *you* think is better for sentiment analysis?

**Answer:** I have some mixed feelings about which model shoul perform better. On one hand, NN-based models perform better in many classes of models, but tree-based models perform not so good texts. On the other hand, the number of trainning sample we have in this dataset may not be enough to train a Neural Network with good score.

### (TODO) More testing

We now have a trained model which has been deployed and which we can send processed reviews to and which returns the predicted sentiment. However, ultimately we would like to be able to send our model an unprocessed review. That is, we would like to send the review itself as a string. For example, suppose we wish to send the following review to our model.

In [108]:
test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'

The question we now need to answer is, how do we send this review to our model?

Recall in the first section of this notebook we did a bunch of data processing to the IMDb dataset. In particular, we did two specific things to the provided reviews.
 - Removed any html tags and stemmed the input
 - Encoded the review as a sequence of integers using `word_dict`
 
In order process the review we will need to repeat these two steps.

**TODO**: Using the `review_to_words` and `convert_and_pad` methods from section one, convert `test_review` into a numpy array `test_data` suitable to send to our model. Remember that our model expects input of the form `review_length, review[500]`.

In [109]:
# TODO: Convert test_review into a form usable by the model and save the results in test_data
test_data = review_to_words(test_review)
test_data

['simplest',
 'pleasur',
 'life',
 'best',
 'film',
 'one',
 'combin',
 'rather',
 'basic',
 'storylin',
 'love',
 'adventur',
 'movi',
 'transcend',
 'usual',
 'weekend',
 'fair',
 'wit',
 'unmitig',
 'charm']

In [110]:
test_converted = convert_and_pad_data( word_dict, [test_data])
test_converted

(array([[   1, 1374,   50,   53,    3,    4,  878,  173,  392,  682,   29,
          724,    2, 4425,  275, 2079, 1061,  760,    1,  582,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0

In [111]:
test_converted[0].shape

(1, 500)

Now that we have processed the review, we can send the resulting array to our model to predict the sentiment of the review.

In [112]:
pred1 = predictor.predict(test_converted[0])

In [113]:
pred1

array(0.9205246, dtype=float32)

In [114]:
np.round(pred1)

1.0

In [120]:
predictor.delete_endpoint()

## Checking the predict_fn

Since the return value of our model is close to `1`, we can be certain that the review we submitted is positive.

### Delete the endpoint

Of course, just like in the XGBoost notebook, once we've deployed an endpoint it continues to run until we tell it to shut down. Since we are done using our endpoint for now, we can delete it.

In [121]:
#estimator.delete_endpoint()

## Step 6 (again) - Deploy the model for the web app

Now that we know that our model is working, it's time to create some custom inference code so that we can send the model a review which has not been processed and have it determine the sentiment of the review.

As we saw above, by default the estimator which we created, when deployed, will use the entry script and directory which we provided when creating the model. However, since we now wish to accept a string as input and our model expects a processed review, we need to write some custom inference code.

We will store the code that we write in the `serve` directory. Provided in this directory is the `model.py` file that we used to construct our model, a `utils.py` file which contains the `review_to_words` and `convert_and_pad` pre-processing functions which we used during the initial data processing, and `predict.py`, the file which will contain our custom inference code. Note also that `requirements.txt` is present which will tell SageMaker what Python libraries are required by our custom inference code.

When deploying a PyTorch model in SageMaker, you are expected to provide four functions which the SageMaker inference container will use.
 - `model_fn`: This function is the same function that we used in the training script and it tells SageMaker how to load our model.
 - `input_fn`: This function receives the raw serialized input that has been sent to the model's endpoint and its job is to de-serialize and make the input available for the inference code.
 - `output_fn`: This function takes the output of the inference code and its job is to serialize this output and return it to the caller of the model's endpoint.
 - `predict_fn`: The heart of the inference script, this is where the actual prediction is done and is the function which you will need to complete.

For the simple website that we are constructing during this project, the `input_fn` and `output_fn` methods are relatively straightforward. We only require being able to accept a string as input and we expect to return a single value as output. You might imagine though that in a more complex application the input or output may be image data or some other binary data which would require some effort to serialize.

### (TODO) Writing inference code

Before writing our custom inference code, we will begin by taking a look at the code which has been provided.

In [2]:
!pygmentize serve/predict.py

import argparse
import json
import os
import pickle
import sys
import sagemaker_containers
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

from model import LSTMClassifier

from utils import review_to_words, convert_and_pad, convert_and_pad_data

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model (predict.py).")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LSTMClassifier(model_info['embedding_dim'], model_info['hidden_dim'], model_info['vocab_size'])

    # Load the store model parameters

As mentioned earlier, the `model_fn` method is the same as the one provided in the training code and the `input_fn` and `output_fn` methods are very simple and your task will be to complete the `predict_fn` method. Make sure that you save the completed file as `predict.py` in the `serve` directory.

**TODO**: Complete the `predict_fn()` method in the `serve/predict.py` file.

### Deploying the model

Now that the custom inference code has been written, we will create and deploy our model. To begin with, we need to construct a new PyTorchModel object which points to the model artifacts created during training and also points to the inference code that we wish to use. Then we can call the deploy method to launch the deployment container.

**NOTE**: The default behaviour for a deployed PyTorch model is to assume that any input passed to the predictor is a `numpy` array. In our case we want to send a string so we need to construct a simple wrapper around the `RealTimePredictor` class to accomodate simple strings. In a more complicated situation you may want to provide a serialization object, for example if you wanted to sent image data.

In [100]:
estimator.model_data

's3://sagemaker-us-east-2-680176548196/sagemaker-pytorch-190528-1311-029-2cfdf7de/output/model.tar.gz'

In [77]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)

predictor = model.deploy(initial_instance_count=1, instance_type=use_instance)

---------------------------------------------------------------------------------------------------!

In [78]:
predictor

### Testing the model

Now that we have deployed our model with the custom inference code, we should test to see if everything is working. Here we test our model by loading the first `250` positive and negative reviews and send them to the endpoint, then collect the results. The reason for only sending some of the data is that the amount of time it takes for our model to process the input and then perform inference is quite long and so testing the entire data set would be prohibitive.

In [75]:
import glob

def test_reviews(data_dir='../data/aclImdb', stop=250, predictor=None):
    
    results = []
    ground = []
    
    # We make sure to test both positive and negative reviews    
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterate through the files and send them to the predictor
        for f in files:
            with open(f) as review:
                # First, we store the ground truth (was the review positive or negative)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Read in the review and convert to 'utf-8' for transmission via HTTP
                review_input = review.read().encode('ascii', errors='ignore')
                print(review_input)
                # Send the review to the predictor and store the results
                review_output = predictor.predict(review_input).decode('ascii')
                print(review_output)
                results.append(round(float(review_output)))
                
            # Sending reviews to our endpoint one at a time takes a while so we
            # only send a small number of reviews
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

In [79]:
ground, results = test_reviews(predictor=predictor)

Starting  pos  files
b"Although flawed in it's view of homosexuals, this movie will shed light for the viewer about the myths and inaccuracies concerning AIDS. Despite the depressing subject matter, this film depicts a warm friendship between two boys, and will make you laugh as well as cry. Very well-acted by all, especially Joseph Mazzello and Brad Renfro. The language is a little strong, though appropriate, and it's an entertaining and intelligent film for the whole family. But remember to have the Kleenex ready!"
1.0
b"I saw this black and white comedy noir yesterday at the London film Festival. Structurally, it has been compared to Pulp Fiction but it is perhaps closer to the structure of Amores Perros and the slacker mood of Kevin Smith's Clerks. Four stories intersect at a French motorway diner. The first vignette has Franck (Edouard Baer) bungling a hold up at the diner. The waitress, Suzie (Anna Mouglalis) takes pity and tell him her story. The second has two incompetent kidna

1.0
b'"The Desperadoes" (1943) is a genuine classic, not for its story (which is fairly routine), but for its technical production elements. This was a landmark western, the biggest ever at the time of its release and all the more unique because it was a Columbia production-a lightweight studio with a bottom feeding reputation. Only Fox\'s "Jesse James" (also starring Randolph Scott) from a few years earlier gave anywhere near this lavish a treatment to the genre. Although it would be eclipsed in a few years by "The Searchers" and "High Noon", "The Desperadoes" was a ground breaking effort and a historical treasure. <br /><br />In 1863, the economy in the town of Red Valley, Utah is based on rounding up and selling wild horses to the Union Army. The script gets a little messed up here with references to the railroad (which was several years away in Utah\'s future) and Custer\'s Last Stand (Custer was busy fighting Stuart in Pennsylvania at the time) but these are not important plot ele

1.0
b'We just finished screening El Padrino in Australia. A phenomenal piece of film work. We look forward to seeing many more films from Mr. Chapa in the future. It was wonderful to see such a well put together film with such suspense and a story that shall remain an instant classic. Seeing a film with great quality truly outlines Chapa\'s serious potential and his adept skill as a writer, actor, director, and filmmaker. Chapa has impressed many with his triumphant performance in "blood in and blood out" and now he has proved to all who have see his works his potential to become a critically acclaimed film maker with genuine artistic control. With his lead role Kilo Vasquez being a perfect combination between Milo Velka from "Blood in Blood Out" and Al Pacino from "Scarface" the film will do wonders for us here in Australia.'
1.0
b'After a man turns up dead, a soldier becomes the prime suspect. Undoubtedly the best film to feature three Roberts and all of them are in fine form. Young 

1.0
b"I really liked this one. (SPOILERS??) It had a really good plot, the main female in this movie is really kewl.<br /><br />Despite the fact that she's the only one left alive and her lover dead, it seemed to be much like Ninja Scroll. Another kick ass movie. ;)<br /><br />Watch it in japanese with subtitles. I don't know where the idiots who learned to speak english are, but for some reason all dubs get an F in translation techniques. The subtitles are more correct.<br /><br />9/10<br /><br />Quality: 7/10 Entertainment: 10/10 Replayable: 9/10"
1.0
b'The main achievement of this film is that though racially unipolar, the film still manages to carve out a tableaux of war portrayals that leave a lasting identification with whoever may view it, and whoever was present at this time. Though good films may have the ability of universalizing their subjects, which is often a hard thing to do; great films have the ability of universalizing their unipolar subjects, which is what this film d

1.0
b"I expected FAME to be an uplifting film but it ended up the opposite. The overall plot which follows the lives of several determined students attending a performing arts school has strong potential. However, FAME builds its characters up beautifully and then leaves us with so many questions when its over. I was very surprised when the graduation scene pops up -- we thought the DVD had skipped or something. All of the characters have internal and external conflicts of some sort and virtually none of them are resolved when the movie ends! You might think there are too many characters, but its probably too many scenes. Its evident the film was cut up and shortened because its sometimes lacks transition. I think Laura Dean as Lisa Monroe is my favorite character. I really connected with her character's ambition and following her heart. Boyd Gaines as Michael, the stereotypical poor student who can't read but is a divine dancer, is also very good. I didn't especially like Irene Cara's

...

1.0
b"It's certainly a direct-to-video, but the story is not as bad as most of the other reviewers think. I quite like the fact the hero is doing the wrong thing most of the time.<br /><br />The hero's reactions and the reactions of the rebels are just human. The Hopper character is actually playing god. That might be the right thing to do, but one may not like that anyway.<br /><br />In the end, the god player is doomed to death, and the hero, who would have spent his own life, can live. Quite a morale. :-)<br /><br />The most unrealistic thing I saw, is that earth is doing so well with no moon stabilizing its rotation."
1.0
b"The director has no clue. I know ... That is the obvious comment. Maybe, we should delve into the story ... the relationships ... how about the quality of the actors?<br /><br />The story is ... well, idiotic would be a simple yet honest answer.<br /><br />The actors are ... they tried very hard. Can they be faulted for the director's choices?<br /><br />All I c

1.0
b"This movie was horrid and at the end made me wonder why someone went to the trouble to make it. Now it was not all bad, I have studied film and this film was put together very nicely and had very good cinematic everything with interesting angles to very nice lighting and excellent camera work. I wish I could have seen it back in school because it would have made a good film to write a paper on. BUT........ Since I have graduated and lost most of my film pretentiousness I have realized that a film should be entertaining above all, this movie was long and boring and I'm not sure when it finally got to the point that it was worth my time.<br /><br />"
1.0
b"I saw this movie in Blockbuster and thought it might be a good Sunday evening movie when there is nothing else to do. So I bought it used, 3 DVDs for $25. Although it was cheap, my money would have been better spent on a nail to drive through my foot.<br /><br />The film started out nicely. Kinda Dark and mysterious. I was always

0.0
b'it\'s embarrassing I had like 3 minutes on my way to a job to stop at the video store and it was 2 for 1 night and I was really intrigued by the half nekkid pic of the \'star\'. <br /><br />I guess this film shows what the new york film school and sir daddy\'s fortune - judging by the bio of this clown in the lead - can do for you and you and you cause that\'s about what we have here and in addition a photoshopped pic of the lead "actor" with someone else\'s body in a still image that doesn\'t happen anywhere in the movie. it\'s weird cause in so many ways it had money thrown at it obviously low budget money buckets but from the outset when all the extras are laughing in their scene of terror it doesn\'t bode well would have maybe had some charm if it had been done for 2 cents! in short order I skipped scenes and fast forwarded to see the image on the box that was all I really cared about. strange, why don\'t I just rent a porno or something? but wow there is bad acting that\'s f

1.0
b'"A scientist has developed a serum which grotesquely distorts the victim\'s hands and heads. The scientist decides to use his serum on a concert pianist to extort money from him for the cure as well as take the man\'s daughter for a wife," according to the DVD sleeve\'s synopsis. J. Carrol Naish (as Dr. Igor Markoff) plays "The Monster Maker" in the low budget Bela Lugosi mode.<br /><br />Mr. Naish\'s serum causes a real disorder, "acromegaly", which American Heritage defines as, "A chronic disease of adults marked by enlargement of the bones of the extremities, face, and jaw that is caused by overactivity of the pituitary gland." Ralph Morgan (as Anthony Lawrence) plays the afflicted man with some dignity. Pretty blonde Wanda McKay (as Patricia "Pat" Lawrence) is the daughter desired by mad scientist Naish; in early scenes, Ms. McKay and Naish emote hilariously.<br /><br />Watch for Tala Birell (as Maxine) in a surprisingly good supporting performance; she plays the somewhat Gar

0.0
b'If there were an EPA for film, then this movie would get their most sincere approval. If we all recycled our "stuff" to this degree, we\'d never run out of anything.<br /><br />Funny how I was reminded of this movie when I first saw Starwars I: The Phantom Menace. At least Lucas didn\'t recycle his old footage.<br /><br />This is a dud. But it\'s a nice dud. Cute in spots (I liked when the kid said, "damn rocks"). And, if you like explosions (even recycled ones) you will get your fill.<br /><br />Actually for an obviously "no budget" film, it makes out fairly well. Acting is weak, but there is a little characterisation here and there. Story is predictable, but will lead you along anyhow.<br /><br />This is an "everybody chases the kid" type of movie which probably will appeal mostly to younger audiences. I gave this one a 2 out of 10.<br /><br />I dug up my old VHS copy of this film. I don\'t think it\'s on DVD.'
1.0


In [80]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

0.588

As an additional test, we can try sending the `test_review` that we looked at earlier.

In [83]:
test_review

'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'

In [84]:
predictor.predict(test_review)

b'1.0'

Now that we know our endpoint is working as expected, we can set up the web page that will interact with it. If you don't have time to finish the project now, make sure to skip down to the end of this notebook and shut down your endpoint. You can deploy it again when you come back.

## Step 7 (again): Use the model for the web app

> **TODO:** This entire section and the next contain tasks for you to complete, mostly using the AWS console.

So far we have been accessing our model endpoint by constructing a predictor object which uses the endpoint and then just using the predictor object to perform inference. What if we wanted to create a web app which accessed our model? The way things are set up currently makes that not possible since in order to access a SageMaker endpoint the app would first have to authenticate with AWS using an IAM role which included access to SageMaker endpoints. However, there is an easier way! We just need to use some additional AWS services.

<img src="Web App Diagram.svg">

The diagram above gives an overview of how the various services will work together. On the far right is the model which we trained above and which is deployed using SageMaker. On the far left is our web app that collects a user's movie review, sends it off and expects a positive or negative sentiment in return.

In the middle is where some of the magic happens. We will construct a Lambda function, which you can think of as a straightforward Python function that can be executed whenever a specified event occurs. We will give this function permission to send and recieve data from a SageMaker endpoint.

Lastly, the method we will use to execute the Lambda function is a new endpoint that we will create using API Gateway. This endpoint will be a url that listens for data to be sent to it. Once it gets some data it will pass that data on to the Lambda function and then return whatever the Lambda function returns. Essentially it will act as an interface that lets our web app communicate with the Lambda function.

### Setting up a Lambda function

The first thing we are going to do is set up a Lambda function. This Lambda function will be executed whenever our public API has data sent to it. When it is executed it will receive the data, perform any sort of processing that is required, send the data (the review) to the SageMaker endpoint we've created and then return the result.

#### Part A: Create an IAM Role for the Lambda function

Since we want the Lambda function to call a SageMaker endpoint, we need to make sure that it has permission to do so. To do this, we will construct a role that we can later give the Lambda function.

Using the AWS Console, navigate to the **IAM** page and click on **Roles**. Then, click on **Create role**. Make sure that the **AWS service** is the type of trusted entity selected and choose **Lambda** as the service that will use this role, then click **Next: Permissions**.

In the search box type `sagemaker` and select the check box next to the **AmazonSageMakerFullAccess** policy. Then, click on **Next: Review**.

Lastly, give this role a name. Make sure you use a name that you will remember later on, for example `LambdaSageMakerRole`. Then, click on **Create role**.

#### Part B: Create a Lambda function

Now it is time to actually create the Lambda function.

Using the AWS Console, navigate to the AWS Lambda page and click on **Create a function**. When you get to the next page, make sure that **Author from scratch** is selected. Now, name your Lambda function, using a name that you will remember later on, for example `sentiment_analysis_func`. Make sure that the **Python 3.6** runtime is selected and then choose the role that you created in the previous part. Then, click on **Create Function**.

On the next page you will see some information about the Lambda function you've just created. If you scroll down you should see an editor in which you can write the code that will be executed when your Lambda function is triggered. In our example, we will use the code below. 

```python
# We need to use the low-level library to interact with SageMaker since the SageMaker API
# is not available natively through Lambda.
import boto3

def lambda_handler(event, context):

    # The SageMaker runtime is what allows us to invoke the endpoint that we've created.
    runtime = boto3.Session().client('sagemaker-runtime')

    # Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
    response = runtime.invoke_endpoint(EndpointName = '**ENDPOINT NAME HERE**',    # The name of the endpoint we created
                                       ContentType = 'text/plain',                 # The data format that is expected
                                       Body = event['body'])                       # The actual review

    # The response is an HTTP response whose body contains the result of our inference
    result = response['Body'].read().decode('utf-8')

    return {
        'statusCode' : 200,
        'headers' : { 'Content-Type' : 'text/plain', 'Access-Control-Allow-Origin' : '*' },
        'body' : result
    }
```

Once you have copy and pasted the code above into the Lambda code editor, replace the `**ENDPOINT NAME HERE**` portion with the name of the endpoint that we deployed earlier. You can determine the name of the endpoint using the code cell below.

In [85]:
predictor.endpoint

'sagemaker-pytorch-2019-05-28-12-19-49-431'

Once you have added the endpoint name to the Lambda function, click on **Save**. Your Lambda function is now up and running. Next we need to create a way for our web app to execute the Lambda function.

### Setting up API Gateway

Now that our Lambda function is set up, it is time to create a new API using API Gateway that will trigger the Lambda function we have just created.

Using AWS Console, navigate to **Amazon API Gateway** and then click on **Get started**.

On the next page, make sure that **New API** is selected and give the new api a name, for example, `sentiment_analysis_api`. Then, click on **Create API**.

Now we have created an API, however it doesn't currently do anything. What we want it to do is to trigger the Lambda function that we created earlier.

Select the **Actions** dropdown menu and click **Create Method**. A new blank method will be created, select its dropdown menu and select **POST**, then click on the check mark beside it.

For the integration point, make sure that **Lambda Function** is selected and click on the **Use Lambda Proxy integration**. This option makes sure that the data that is sent to the API is then sent directly to the Lambda function with no processing. It also means that the return value must be a proper response object as it will also not be processed by API Gateway.

Type the name of the Lambda function you created earlier into the **Lambda Function** text entry box and then click on **Save**. Click on **OK** in the pop-up box that then appears, giving permission to API Gateway to invoke the Lambda function you created.

The last step in creating the API Gateway is to select the **Actions** dropdown and click on **Deploy API**. You will need to create a new Deployment stage and name it anything you like, for example `prod`.

You have now successfully set up a public API to access your SageMaker model. Make sure to copy or write down the URL provided to invoke your newly created public API as this will be needed in the next step. This URL can be found at the top of the page, highlighted in blue next to the text **Invoke URL**.

## Step 4: Deploying our web app

Now that we have a publicly available API, we can start using it in a web app. For our purposes, we have provided a simple static html file which can make use of the public api you created earlier.

In the `website` folder there should be a file called `index.html`. Download the file to your computer and open that file up in a text editor of your choice. There should be a line which contains **\*\*REPLACE WITH PUBLIC API URL\*\***. Replace this string with the url that you wrote down in the last step and then save the file.

Now, if you open `index.html` on your local computer, your browser will behave as a local web server and you can use the provided site to interact with your SageMaker model.

If you'd like to go further, you can host this html file anywhere you'd like, for example using github or hosting a static site on Amazon's S3. Once you have done this you can share the link with anyone you'd like and have them play with it too!

> **Important Note** In order for the web app to communicate with the SageMaker endpoint, the endpoint has to actually be deployed and running. This means that you are paying for it. Make sure that the endpoint is running when you want to use the web app but that you shut it down when you don't need it, otherwise you will end up with a surprisingly large AWS bill.

**TODO:** Make sure that you include the edited `index.html` file in your project submission.

Now that your web app is working, trying playing around with it and see how well it works.

**Question**: Give an example of a review that you entered into your web app. What was the predicted sentiment of your example review?

**Answer:** I tried first to give a short test review, like "excellent movie" and to my surprise the response wa "negative". Then I copy pasted a review from `terst_review` variable, and it turned positive as expected. I suppose the model is not good enough on short reviews.

### Delete the endpoint

Remember to always shut down your endpoint if you are no longer using it. You are charged for the length of time that the endpoint is running so if you forget and leave it on you could end up with an unexpectedly large bill.

In [86]:
predictor.delete_endpoint()